In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_con = pd.read_csv('../데이터자료/1. 상장폐지종목 정리끝.csv', encoding='cp949', index_col=0)
df_invis = pd.read_csv('../데이터자료/1. 코스닥 기업 개별재무제표.csv', encoding='cp949')


In [ ]:
df_con.columns

In [ ]:
df_con = df_con[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','[제조]매출액증가율(IFRS연결)',
'[제조]자기자본순이익률(IFRS연결)','[제조]매출액순이익률(IFRS연결)','[제조]유동비율(IFRS연결)','[제조]자기자본구성비율(IFRS연결)',
'[제조]총자본회전률(IFRS연결)','[제조]자기자본회전률(IFRS연결)','[제조]총자본투자효율(IFRS연결)','[제조]설비투자효율(IFRS연결)',
'[제조]순이익증가율(IFRS연결)']]

In [ ]:
df_invis.columns

In [ ]:
df_invis=df_invis[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','[제조]매출액증가율(IFRS)',
'[제조]자기자본순이익률(IFRS)','[제조]매출액순이익률(IFRS)','[제조]유동비율(IFRS)','[제조]자기자본구성비율(IFRS)',
'[제조]총자본회전률(IFRS)','[제조]자기자본회전률(IFRS)','[제조]총자본투자효율(IFRS)','[제조]설비투자효율(IFRS)',
'[제조]순이익증가율(IFRS)']]

In [ ]:

# 데이터 기본 정보 확인
df_con.info()
print(df_con.head(20))
df_invis.info()
print(df_invis.head(20))

# 컬럼명 일치를 위한 컬럼명 변경
df_con.columns = df_con.columns.str.replace('연결', '')

# 누락된 데이터 채우기
fill_columns = df_con.columns[df_con.columns.str.contains("\[제조\]")]
for col in fill_columns:
    df_con[col] = df_con[col].fillna(df_invis[col])

# 누락된 데이터가 있는 행 제거
df_con.dropna(subset=fill_columns, inplace=True)

# 상장폐지일이 존재하면서 소속코드가 5인 데이터 처리
df_con.loc[(~df_con["상장폐지일"].isnull()) & (df_con["소속코드"] == 5), "상장폐지일"] = np.NaN

# 컬럼명 간소화
df_con.columns = df_con.columns.str.replace('[\[제조\]\(\)IFRS]', '', regex=True)

# 최종 데이터 저장
df_con.to_csv('../데이터자료/2. 연결개별통합_리팩토링.csv', index=False)

In [1]:
import pandas as pd
import numpy as np
# 데이터 로드
df_1 = pd.read_csv('../데이터자료/0. 연결재무제표.csv', encoding='cp949')
df_2 = pd.read_csv('../데이터자료/0. 코스닥기업 연결재무제표.csv', encoding='cp949')


In [3]:
df_1=df_1[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','[제조]매출액증가율(IFRS연결)',
'[제조]자기자본순이익률(IFRS연결)','[제조]매출액순이익률(IFRS연결)','[제조]유동비율(IFRS연결)','[제조]자기자본구성비율(IFRS연결)',
'[제조]총자본회전률(IFRS연결)','[제조]자기자본회전률(IFRS연결)','[제조]총자본투자효율(IFRS연결)','[제조]설비투자효율(IFRS연결)',
'[제조]순이익증가율(IFRS연결)']]

In [5]:
df_2=df_2[['회사명', '거래소코드', '회계년도', '[제조]매출액증가율(IFRS연결)',
'[제조]자기자본순이익률(IFRS연결)','[제조]매출액순이익률(IFRS연결)','[제조]유동비율(IFRS연결)','[제조]자기자본구성비율(IFRS연결)',
'[제조]총자본회전률(IFRS연결)','[제조]자기자본회전률(IFRS연결)','[제조]총자본투자효율(IFRS연결)','[제조]설비투자효율(IFRS연결)',
'[제조]순이익증가율(IFRS연결)']]

In [7]:



# 데이터 기본 정보 확인
df_2.info()
df_1.info()

# 상장폐지된 데이터 필터링
df_default = df_1[~df_1["상장폐지일"].isnull()]
print("상장폐지 기업 수:", len(df_default["회사명"].unique()))

# 2022년도 상장폐지 데이터 처리
df_default_2022 = df_default[df_default["상장폐지일"].str.contains('2022', na=False)]
for idx in df_default_2022.index:
    df_default.loc[idx, "상장폐지일"] = np.NaN
    df_default.loc[idx, "소속코드"] = 5

# 상장폐지 사유 데이터 처리
df_default_mod = pd.read_csv('../데이터자료/1. 상장폐지종목_사유등록.csv', encoding='cp949', index_col=0)
df = pd.concat([df_1, df_default_mod["상장폐지사유"]], axis=1)

# 코넥스, 합병 관련 데이터 제거
df = df[~df["상장폐지사유"].str.contains('코넥스', na=False)]
df = df[~df["상장폐지사유"].str.contains('합병', na=False)]

# 상장폐지 사유가 있는 데이터의 소속코드를 5로 수정
df.loc[~df["상장폐지사유"].isnull(), "소속코드"] = 5

# 날짜 데이터 처리 및 필터링
df["상장일"] = pd.to_datetime(df['상장일'])
df["상장폐지일"] = pd.to_datetime(df['상장폐지일'])
df["회계년도"] = pd.to_datetime(df["회계년도"])
df = df[df["상장폐지일"].dt.year >= 2011]

# 회계년도 기준 필터링
df["회계월"] = df["회계년도"].dt.month
df = df[df["회계월"] == 12]

# 외국계 기업 제거
df = df[df["거래소코드"] <= 900000]

# 중복 데이터 확인 및 제거
df["상장년도"] = df["상장일"].dt.year
df["회계년"] = df["회계년도"].dt.year
df["연차"] = df["회계년"] - df["상장년도"]
df = df[df["연차"] > 0]

b = df[["회사명", "거래소코드"]]
d = b.groupby("회사명")["거래소코드"].unique()
for company, codes in d.items():
    if len(codes) > 1:
        df = df[~df["거래소코드"].isin(codes)]

# 인덱스 초기화 및 최종 데이터 저장
df.reset_index(drop=True, inplace=True)
df.to_csv('../데이터자료/상장폐지종목 정리끝1.csv', encoding='cp949')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14347 entries, 0 to 14346
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   회사명                   14347 non-null  object 
 1   거래소코드                 14347 non-null  int64  
 2   회계년도                  14347 non-null  object 
 3   [제조]매출액증가율(IFRS연결)    9016 non-null   float64
 4   [제조]자기자본순이익률(IFRS연결)  9016 non-null   float64
 5   [제조]매출액순이익률(IFRS연결)   9016 non-null   float64
 6   [제조]유동비율(IFRS연결)      9016 non-null   float64
 7   [제조]자기자본구성비율(IFRS연결)  9016 non-null   float64
 8   [제조]총자본회전률(IFRS연결)    9016 non-null   float64
 9   [제조]자기자본회전률(IFRS연결)   9016 non-null   float64
 10  [제조]총자본투자효율(IFRS연결)   9016 non-null   float64
 11  [제조]설비투자효율(IFRS연결)    9016 non-null   float64
 12  [제조]순이익증가율(IFRS연결)    9016 non-null   float64
dtypes: float64(10), int64(1), object(2)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14347 entries

C:\Users\LG\AppData\Local\Temp\ipykernel_14080\3553973272.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["회계년도"] = pd.to_datetime(df["회계년도"])
